# Retrieval-Augmented Generation (RAG)

* [Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks](https://arxiv.org/abs/2005.11401)
* [RAG From Scratch](https://github.com/langchain-ai/rag-from-scratch?tab=readme-ov-file)
* [12 RAG Pain Points and Solutions](https://originshq.com/blog/12-rag-pain-points-and-solutions/)
* [Seven Failure Points When Engineering a Retrieval Augmented Generation System](https://arxiv.org/pdf/2401.05856)
* [Архитектура RAG: полный гайд](https://habr.com/ru/amp/publications/791034/)
* [RAG (Retrieval Augmented Generation) — простое и понятное объяснение](https://habr.com/ru/articles/779526/)

## Интро

RAG - это техника повышения точности и надежности LLM с использованием документов, полученных из внешних источников. 

Процесс генерации информации при помощи RAG выглядит следующим образом:
1. Пользователь составляет **запрос**
2. Запрос отправляется в **поисковик**
    * Определяется процесс поиска и извлечения документов по запросу:
        * **Retrievals**
        * **Векторная база данных**
3. Извлеченные документы отправляются в **генератор**
    * Комбинация входного запроса и извлеченных документов
4. Итоговый ответ

## Векторная база данных

Векторные базы данных хранят векторы контекста (эмбеддинги). На основе такой базы знаний производится **семантический поиск**, который помогает определить, насколько документы релевантны запросу.

1. Правильно нарезаем базу знаний на чанки для базы данных
    * По символам? По абзацам? Логически? С перекрытием?
    * Важные параметры: количество чанков, размер чанка
2. Какой эмбедер выбрать
    * Нужно ли его дообучать?
    
Примеры векторных баз:
* Pinecone
* ChromaDB
* Weaviate
* ElasticSearch

## Retrievals

* Стандартный ретривер - косинусная близость вектора запроса и вектора документа
* BM25 ретривер
* TF‑IDF ретривер
* KNN ретривер

[Ретриверы из langchain_community](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/retrievers)

## MultiQuery

Перефразирование запроса несколько раз, извлечение документов, затем обработка всего пула документов

## Дообучение LLM

В задаче RAG на вход LLM поступают пары сущностей: входной запрос и контекст, состоящий из документов.

LLM можно натюнить на такой вход.

## Промпт-инженеринг

### Промпт для обработки первоначального запроса пользователя

```
Вот запрос пользователя. Думай по шагам и оцени, что пользователь хотел узнать.
В качестве ответа выведи настоящий запрос, который хотел ввести пользователь.
```

### Промпт для суммаризации документа

Если по запросу найдено много длинных чанков, можно попосить LLM суммаризовать каждый из нейденных чанков.

### Корректный системный промпт

Источник: [GigaChain работа с промптами](https://github.com/ai-forever/gigachain/tree/master/hub/prompts)

```
input_variables: []
output_parser: null
template: 'Ты - система информационного поиска. Тебе дан вопрос и релевантные отрывки текста из нескольких документов.
Создай краткий и информативный ответ (не более 150 слов) на заданный вопрос,
основываясь исключительно на приведенных отрывках документов. Ты должна использовать только информацию из приведенных отрывков.
Используй непредвзятый и журналистский тон. Не повторяй текст.
Создай окончательный ответ ("FINAL ANSWER").
Не пытайся придумать ответ.
Отвечай только на русском языке за исключением специальных терминов.
Если документы не содержат ответа на вопрос, скажи, что "Я не могу ответить на вопрос на основе информации. Попробуйте переформулировать вопрос."
template_format: f-string
_type: prompt
```

### Пользовательский промпт

```
input_variables: [question, summaries]
output_parser: null
template: "QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:"
template_format: f-string
_type: prompt
```

## Метрики оценки RAG-системы

1. **Golden set** (вопросы и ответы) должен быть составлен при помощи естественного интеллекта. Лучше всего, если будет несколько экземпляров ответов на один вопрос.
2. В задаче важно измерять отдельно:
    * Метрики поиска (ранжирования)
    * Метрики генерации
        * ROUGE, BERTScore, BLEURT, METEOR. Можно написать взвешенную сумму перечисленных метрик.